<a href="https://colab.research.google.com/github/Abonia1/LLM-finetuning/blob/main/Document_Search_%2B_Faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import os
import pandas as pd
import faiss
import time
import numpy as np
from sentence_transformers import SentenceTransformer

In [4]:
url = "https://github.com/franciscadias/data/blob/master/abcnews-date-text.csv"
url = "https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv#"

df = pd.read_csv(url)
df.head(5)



,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [5]:
data=df.headline_text.to_list()

In [6]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)

In [7]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

In [ ]:
index.add_with_ids(encoded_data, np.array(range(0, len(data))))

In [8]:
faiss.write_index(index, 'abc_news')

In [9]:

index = faiss.read_index('abc_news')

In [10]:
def search(query):
   t=time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('totaltime: {}'.format(time.time()-t))
   return [data[_id] for _id in top_k[1].tolist()[0]]

In [11]:
query=str(input())
results=search(query)
print('results :')
for result in results:
   print('\t',result)

global temperature
totaltime: 0.039696693420410156
results :
	 youtube stunt death foreshadowed by tweet
	 youtube stunt death foreshadowed by tweet
	 youtube stunt death foreshadowed by tweet
	 youtube stunt death foreshadowed by tweet
	 youtube stunt death foreshadowed by tweet
